In [1]:
import requests, json
import sqlite3
%load_ext sql

In [3]:
url = 'https://tradestie.com/api/v1/apps/reddit?date=2022-04-03'
response = requests.get(url).text


In [4]:
json.loads(response)

[{'no_of_comments': 131,
  'sentiment': 'Bullish',
  'sentiment_score': 0.01,
  'ticker': 'GME'},
 {'no_of_comments': 90,
  'sentiment': 'Bullish',
  'sentiment_score': 0.039,
  'ticker': 'AMC'},
 {'no_of_comments': 25,
  'sentiment': 'Bullish',
  'sentiment_score': 0.053,
  'ticker': 'TSLA'},
 {'no_of_comments': 21,
  'sentiment': 'Bullish',
  'sentiment_score': 0.442,
  'ticker': 'WISH'},
 {'no_of_comments': 12,
  'sentiment': 'Bearish',
  'sentiment_score': -0.061,
  'ticker': 'QQQ'},
 {'no_of_comments': 10,
  'sentiment': 'Bullish',
  'sentiment_score': 0.104,
  'ticker': 'TV'},
 {'no_of_comments': 9,
  'sentiment': 'Bullish',
  'sentiment_score': 0.014,
  'ticker': 'HMHC'},
 {'no_of_comments': 8,
  'sentiment': 'Bullish',
  'sentiment_score': 0.166,
  'ticker': 'PAA'},
 {'no_of_comments': 6,
  'sentiment': 'Bullish',
  'sentiment_score': 0.095,
  'ticker': 'OXY'},
 {'no_of_comments': 6,
  'sentiment': 'Bullish',
  'sentiment_score': 0.091,
  'ticker': 'AA'},
 {'no_of_comments': 5,

In [5]:
data = json.loads(response)

In [6]:
conn = sqlite3.connect('stockcm.db')
cur = conn.cursor()

In [7]:
data[0].keys()

dict_keys(['no_of_comments', 'sentiment', 'sentiment_score', 'ticker'])

In [8]:
cur.execute('''CREATE TABLE STOCKCM (
    no_of_comments INTEGER,
    sentiment VARCHAR,
    sentiment_score DOUBLE,
    ticker VARCHAR
)''')

In [9]:
query = 'INSERT INTO STOCKCM VALUES(?,?,?,?)'
for i in data:
    cur.execute(query, (i['no_of_comments'], i['sentiment'], i['sentiment_score'], i['ticker']))
conn.commit()

In [10]:
%sql sqlite:///stockcm.db

In [11]:
%%sql
SELECT * FROM STOCKCM LIMIT 10;

 * sqlite:///stockcm.db
Done.


no_of_comments,sentiment,sentiment_score,ticker
131,Bullish,0.01,GME
90,Bullish,0.039,AMC
25,Bullish,0.053,TSLA
21,Bullish,0.442,WISH
12,Bearish,-0.061,QQQ
10,Bullish,0.104,TV
9,Bullish,0.014,HMHC
8,Bullish,0.166,PAA
6,Bullish,0.095,OXY
6,Bullish,0.091,AA


In [ ]:
%%sql
SELECT COUNT(*) FROM STOCKCM;

 * sqlite:///stockcm.db
Done.


COUNT(*)
50


In [12]:
%%sql
SELECT sentiment, count(*)
FROM STOCKCM
GROUP BY sentiment;

 * sqlite:///stockcm.db
Done.


sentiment,count(*)
None,1
Bearish,16
Bullish,33


In [16]:
%%sql
DELETE FROM STOCKCM
WHERE sentiment IS NULL;

 * sqlite:///stockcm.db
1 rows affected.


[]

In [22]:
%%sql
SELECT sentiment, ROUND(avg(sentiment_score), 2) AVERAGE_SCORE
FROM STOCKCM
GROUP BY sentiment;

 * sqlite:///stockcm.db
Done.


sentiment,AVERAGE_SCORE
Bearish,-0.23
Bullish,0.31


In [23]:
%%sql

SELECT SUM(no_of_comments)
FROM STOCKCM
WHERE sentiment = 'Bearish'

 * sqlite:///stockcm.db
Done.


SUM(no_of_comments)
59


In [25]:
%%sql
SELECT * FROM STOCKCM
WHERE sentiment_score = (SELECT MAX(sentiment_score) FROM STOCKCM);

 * sqlite:///stockcm.db
Done.


no_of_comments,sentiment,sentiment_score,ticker
2,Bullish,0.834,NH


In [30]:
%%sql
SELECT sentiment, COUNT(ticker)
FROM STOCKCM
GROUP BY sentiment
HAVING no_of_comments > 5 ORDER BY no_of_comments DESC;

 * sqlite:///stockcm.db
Done.


sentiment,COUNT(ticker)
Bullish,33
Bearish,16
